# Maze and path finding

For searching, we have Depth-First Search, Breadth-First Search, Dijkstra, Best-First Search and A\* algorithms.

Using searching to solve a problem, we should know the target doesn't only means a specific location on the graph, but also can be a state reached during the searching process. Which means even you work on the same location, but it's not the same node in abstraction since you are in different state.

In [36]:
import collections
import heapq
class Solution(object):
    def shortestPathAllKeys(self, grid):
        M = len(grid)
        N = len(grid[0])
        
        location = {v: (m, n) 
                    for m, row in enumerate(grid)
                    for n, v in enumerate(row) 
                    if v not in '.#'}
    
        def neighbors(m, n):
            for nm, nn in ((m-1, n), (m+1, n), (m, n-1), (m, n+1)):
                if 0<=nm<M and 0<=nn<N:
                    yield nm, nn
    
        def bfs_from(source):
            m, n = location[source]
            visited = [[False]*N for _ in range(M)]
            visited[m][n] = True
            queue = [(m, n, 0)]
            dist = {}
            while queue:
                m, n, d = queue.pop(0)
                if grid[m][n] != source and grid[m][n]!= '.':
                    dist[grid[m][n]] = d
                    continue # We don't want a path containing two points
                for nm, nn in neighbors(m, n):
                    if grid[nm][nn] != '#' and not visited[nm][nn]:
                        visited[nm][nn] = True
                        queue.append((nm, nn, d+1))
            return dist
        dists = {place: bfs_from(place) for place in location}
        target_state = 2**sum(p.islower() for p in location)-1
        pq = [(0, '@', 0)]
        cost_sofar = collections.defaultdict(lambda: float('inf'))
        cost_sofar['@', 0] = 0
        while pq:
            d, place, state = heapq.heappop(pq)
            if cost_sofar[place, state] < d: 
                continue
            if state == target_state:
                return d
            
            for nextstation in dists[place]:
                nextd = dists[place][nextstation]
                tempstate = state
                if nextstation.islower():
                    tempstate |= (1<< (ord(nextstation)-ord('a')))
                elif nextstation.isupper():
                    if not (state & (1<< (ord(nextstation)-ord('A')))):
                        continue
                
                if d+nextd < cost_sofar[nextstation, tempstate]:
                    cost_sofar[nextstation, tempstate] = d+nextd
                    heapq.heappush(pq, (d+nextd, nextstation, tempstate))
        return -1
                
    

In [37]:
s = Solution()

In [38]:
grid = ["@.a.#","###.#","b.A.B"]
s.shortestPathAllKeys(grid)

8